# Titanic - Machine Learning from Disaster

- Vamos utilizar os [dados disponíveis no Kaggle](https://www.kaggle.com/competitions/titanic)
    - É um dataset de **competição**
    - O resultado é avaliado através da **acurácia**:
        - _"Sua pontuação é a porcentagem de passageiros que você prevê corretamente. Isso é conhecido como acurácia."_

In [2]:
import pandas as pd

In [3]:
treino = pd.read_csv('train.csv')

In [4]:
teste = pd.read_csv('test.csv')

> Como já houve uma tentativa para esse desafio, vamos apenas replicar algumas etapas e tentar melhorar a precisão de nossa previsão.

In [5]:
# Eliminando as colunas com elevada cardinalidade
treino = treino.drop(['Name','Ticket','Cabin'],axis=1)
teste = teste.drop(['Name','Ticket','Cabin'],axis=1)

In [6]:
# Usando a media para imputar os valores nulos na coluna de idade
treino.loc[treino.Age.isnull(),'Age'] = treino.Age.mean()
teste.loc[teste.Age.isnull(),'Age'] = teste.Age.mean()

In [7]:
# Tratando a coluna Embarked da base de treino usando a moda
treino.loc[treino.Embarked.isnull(),'Embarked'] = treino.Embarked.mode()[0]

In [8]:
# E também a coluna Fare da base de teste usando a média
teste.loc[teste.Fare.isnull(),'Fare'] = teste.Fare.mean()

In [9]:
treino.dtypes=='object'

PassengerId    False
Survived       False
Pclass         False
Sex             True
Age            False
SibSp          False
Parch          False
Fare           False
Embarked        True
dtype: bool

In [10]:
treino.columns[treino.dtypes=='object']

Index(['Sex', 'Embarked'], dtype='object')

In [11]:
treino.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [12]:
treino.Embarked.value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

### Variável Sex

Vamos transformar a coluna Sex código númerico para que a coluna possa ser usada no modelo de aprendizagem

In [13]:
sex = {'male': 1,
       'female': 0}

In [14]:
treino['MaleCheck'] = treino.Sex.apply(lambda x: 1 if x == 'male' else 0)

In [15]:
treino[['Sex', 'MaleCheck']].value_counts()

Sex     MaleCheck
male    1            577
female  0            314
dtype: int64

In [16]:
teste['MaleCheck'] = teste.Sex.apply(lambda x: 1 if x == 'male' else 0)

In [17]:
teste[['Sex', 'MaleCheck']].value_counts()

Sex     MaleCheck
male    1            266
female  0            152
dtype: int64

### OneHotEncoder na coluna Embarked: Por quê

- OneHotEncoder é uma técnica de pré-processamento usada em aprendizado de máquina para lidar com variáveis categóricas. Ele converte variáveis categóricas em uma representação binária, criando uma nova coluna para cada categoria com valores binários indicando a presença ou ausência dessa categoria.

- Essa técnica é especialmente interessante por que há um pequeno número de categorias (3) na coluna 'Embarked', melhorando a eficiencia de aprendizado de máquina, Permitindo que o modelo compreenda e utilize efetivamente informações de variáveis categóricas em seus cálculos.

    - Ao criar colunas binárias para cada local de embarque, o modelo pode interpretar de maneira mais eficaz as informações categóricas, melhorando a capacidade preditiva do algoritmo. Essa abordagem é especialmente relevante quando lidamos com um conjunto de dados contendo um número restrito de categorias, contribuindo para a interpretação e desempenho aprimorados do modelo.

In [18]:
from sklearn.preprocessing import OneHotEncoder

In [19]:
# Criando o encoder
ohe = OneHotEncoder(handle_unknown='ignore',dtype='int32')

In [20]:
ohe = ohe.fit(treino[['Embarked']])

In [21]:
# Fazendo a transformação
ohe.transform(treino[['Embarked']]).toarray()

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0]], dtype=int32)

In [22]:
# Transformando esse resultado em um DataFrame
ohe_df = pd.DataFrame(ohe.transform(treino[['Embarked']]).toarray(),columns=ohe.get_feature_names_out())
ohe_df.head(3)

,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1


In [23]:
# Concatenando o dataframe
treino = pd.concat([treino,ohe_df],axis=1)

In [24]:
# Verificando os valores
treino[['Embarked','Embarked_C','Embarked_Q','Embarked_S']].value_counts()

Embarked  Embarked_C  Embarked_Q  Embarked_S
S         0           0           1             646
C         1           0           0             168
Q         0           1           0              77
dtype: int64

- Agora fazendo o mesmo para a base de teste usando o encoder ohe que criamos acima

In [25]:
ohe_df = pd.DataFrame(ohe.transform(teste[['Embarked']]).toarray(),columns=ohe.get_feature_names_out())

In [26]:
teste = pd.concat([teste,ohe_df],axis=1)

In [27]:
# Verificando os valores
teste[['Embarked','Embarked_C','Embarked_Q','Embarked_S']].value_counts()

Embarked  Embarked_C  Embarked_Q  Embarked_S
S         0           0           1             270
C         1           0           0             102
Q         0           1           0              46
dtype: int64

In [28]:
treino.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,male,22.0,1,0,7.2500,S,1,0,0,1
1,2,1,1,female,38.0,1,0,71.2833,C,0,1,0,0
2,3,1,3,female,26.0,0,0,7.9250,S,0,0,0,1
3,4,1,1,female,35.0,1,0,53.1000,S,0,0,0,1
4,5,0,3,male,35.0,0,0,8.0500,S,1,0,0,1


In [29]:
# Eliminando as colunas que foram tratadas/modificadas
treino = treino.drop(['Sex','Embarked'],axis=1)
teste = teste.drop(['Sex','Embarked'],axis=1)

### Utilizando outros modelos para fazer a previsão
- Foram selecionados algoritmos diferentes dos que testamos nas tentativas anteriores considerando os [outros algoritmos disponíveis no scikit-learn](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning)

- Para essa nova tentativa, deixamos o modelo que conseguiu a melhor acurária para realizar a comparação com outros novos modelos
    - **Regressão Logística**
        - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression
    - **Random Forest**
        - https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier
    - **MLPClassifier (Redes Neurais)**
        - https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier
        

- train_test_split é uma função da biblioteca Scikit-Learn que tem o objetivo de dividir um conjunto de dados em duas partes: uma para treinamento do modelo e outra para avaliação do desempenho do modelo. A função garante que o modelo generalize bem para novos dados não vistos, prevenindo overfitting. O conjunto de treinamento é usado para treinar o modelo, enquanto o conjunto de teste é reservado para avaliação de desempenho.

- Agora, **além do train_test_split**:
    - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
- Vamos usar também o **grid_search** para estimar os melhores parâmetros
    - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
X = treino.drop(['PassengerId','Survived'],axis=1)
# A tabela abaixo atribui o resultado que esperamos, para verificar se há um modelo que possa prever os sobreviventes.
y = treino.Survived

In [32]:
# Separando em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)

### Criando o modelo de teste com **Random Forest Classifier**

In [33]:
from sklearn.ensemble import RandomForestClassifier

In [34]:
# Criando o classificador
# clf -> classificação
# rf -> Random Forest  Classifier
clf_rf = RandomForestClassifier(random_state=42)

In [35]:
# definindo os parâmetros
parametros_rf = {
    'n_estimators': [100,200,500,1000],
    'criterion': ['gini','entropy','log_loss'],
    'max_depth': [2,4,6,8,None],
    'max_features': ['sqrt','log2',None]
}

### Para o **MLP classifier**

In [36]:
from sklearn.neural_network import MLPClassifier

In [37]:
# Criando o classificador
# clf -> classificação
# mlp -> modelo usado (MLP classifier)
clf_mlp = MLPClassifier(random_state=42)

In [38]:
# Definindo os parâmetros
parametros_mlp = {
    'solver':  ['lbfgs','sgd','adam'],
    'alpha': [10.0**(-1),10.0**(-5),10.0**(-7),10.0**(-10)],
    'max_iter': [200,500,1000,5000]
}

### Para a **Regressão Logística**

In [39]:
from sklearn.linear_model import LogisticRegression

In [40]:
# Criando o classificador
# clf -> classificador
# rl -> Regressão Logística
clf_rl = LogisticRegression(random_state=42)

In [41]:
# definindo parametros
parametros_rl = {
    'penalty': ['l1','l2'],
    'C': [0.01,0.1,1,10],
    'solver': ['lbfgs','liblinear','saga'],
    'max_iter': [100,1000,5000,10000]
}

- Fazendo o grid_search

    - O GridSearchCV do scikit-learn é uma técnica de otimização de hiperparâmetros. Ele é usado para encontrar os melhores hiperparâmetros para um modelo de aprendizado de máquina por meio da busca exaustiva em uma grade de parâmetros especificada.

In [43]:
# Importando o KFold e o GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [ ]:
# Para a Regressão Logística
kfold_rl = KFold(shuffle=True,random_state=42,n_splits=8)
grid_search_rl = GridSearchCV(clf_rl, parametros_rl,scoring='accuracy',cv=kfold_rl)
grid_search_rl = grid_search_rl.fit(X_train,y_train)

In [45]:
# Para o RandomForest
kfold_rf = KFold(shuffle=True,random_state=42,n_splits=8)
grid_search_rf = GridSearchCV(clf_rf, parametros_rf,scoring='accuracy',cv=kfold_rf)
grid_search_rf = grid_search_rf.fit(X_train,y_train)

In [ ]:
# Para o MLPClassifier
kfold_mlp = KFold(shuffle=True,random_state=42,n_splits=8)
grid_search_mlp = GridSearchCV(clf_mlp, parametros_mlp,scoring='accuracy',cv=kfold_mlp)
grid_search_mlp = grid_search_mlp.fit(X_train,y_train)

- **Verificando os melhores scores**

In [48]:
# Verificando o melhor score da regressão logística
grid_search_rl.best_score_

0.8019662921348314

In [49]:
# RandomForest
grid_search_rf.best_score_

0.8314606741573034

In [50]:
# MLPClassifier
grid_search_mlp.best_score_

0.8132022471910112

- **E os melhores parâmetros**

In [51]:
# Verificando os melhores parâmetros da regressão logística
grid_search_rl.best_params_

{'C': 0.1, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'lbfgs'}

In [52]:
# Para o RandomForest
grid_search_rf.best_params_

{'criterion': 'entropy',
 'max_depth': 6,
 'max_features': 'sqrt',
 'n_estimators': 200}

In [53]:
# e para o MLPClassifier
grid_search_mlp.best_params_

{'alpha': 0.1, 'max_iter': 500, 'solver': 'adam'}

- **Realizando a previsão nos dados de validação com cada um dos melhores modelos**

In [55]:
# Para a regressão logística
clf_best_rl = grid_search_rl.best_estimator_
y_pred_rl = clf_best_rl.predict(X_val)

In [57]:
# RandomForest
clf_best_rf = grid_search_rf.best_estimator_
y_pred_rf = clf_best_rf.predict(X_val)

In [58]:
# MLPClassifier
clf_best_mlp = grid_search_mlp.best_estimator_
y_pred_mlp = clf_best_mlp.predict(X_val)

### Para finalizar, vamos avaliar esses modelos treinados:
- Para fazer essa análise, será usado:
    - Acurácia (método de avaliação usado na competição):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
    - Matriz de confusão (ajuda a visualizar a distribuição dos erros):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

In [59]:
from sklearn.metrics import accuracy_score

In [60]:
as_rf = accuracy_score(y_val, y_pred_rf) # Verificando acurácia para a árvore de classificação
as_mlp = accuracy_score(y_val, y_pred_mlp) # Verificando acurácia para o knn
as_rl = accuracy_score(y_val, y_pred_rl) # Para a regressão logística

print(as_rf, as_mlp, as_rl)

0.8212290502793296 0.8044692737430168 0.7988826815642458


- Salvando esses valores em um DataFrame para visualizarmos a evolução dos testes

In [61]:
# Criando primeiramente o DataFrame
as_df = pd.DataFrame({
    'modelos': ['random forest.','MLP','reg. log.'],
    'result1': [as_rf,as_mlp,as_rl]
})

as_df

,modelos,result1
0,random forest.,0.821229
1,MLP,0.804469
2,reg. log.,0.798883


### Avaliando a **matriz de confusão**

- Utilizaremos a matriz de confusão afim de validar mais uma vez a ferramenta mais eficiente aplicada ao nosso teste.
    - O módulo confusion_matrix do Scikit-Learn é utilizado para avaliar o desempenho de modelos de classificação. Ele cria uma tabela que mostra a contagem de acertos e erros do modelo em diferentes categorias

In [62]:
from sklearn.metrics import confusion_matrix

In [63]:
# Para a random forest
confusion_matrix(y_val, y_pred_rf)

array([[97,  8],
       [24, 50]])

In [64]:
# Para o MLP (redes neurais)
confusion_matrix(y_val, y_pred_mlp)

array([[90, 15],
       [20, 54]])

In [65]:
# Para a regressão logística
confusion_matrix(y_val, y_pred_rl)

array([[89, 16],
       [20, 54]])

### Fazendo a previsão para os dados de teste
- Vamos usar o modelo com melhor precisão para fazer o predict na base de teste

In [73]:
X_train.head()

,Pclass,Age,SibSp,Parch,Fare,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
331,1,45.5,0,0,28.5000,1,0,0,1
733,2,23.0,0,0,13.0000,1,0,0,1
382,3,32.0,0,0,7.9250,1,0,0,1
704,3,26.0,1,0,7.8542,1,0,0,1
813,3,6.0,4,2,31.2750,0,0,0,1


In [74]:
teste.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,1,0,1,0
1,893,3,47.0,1,0,7.0000,0,0,0,1
2,894,2,62.0,0,0,9.6875,1,0,1,0
3,895,3,27.0,0,0,8.6625,1,0,0,1
4,896,3,22.0,1,1,12.2875,0,0,0,1


In [75]:
rf = teste.copy()
mlp = teste.copy()
rl = teste.copy()

In [83]:
X_teste = teste.drop('PassengerId',axis=1)

In [84]:
# Utilizando todos os modelos para enviar no teste kaggle
y_pred = clf_best_rf.predict(X_teste)

In [85]:
teste['Survived'] = y_pred


In [86]:
base_envio = teste[['PassengerId','Survived']]

In [87]:
base_envio.to_csv('random_forest.csv',index=False)